# The urban geography of inequalities
## Data used
### WEkEO datasets
Dataset queries were generated by using the [WEkEO online platform](https://www.wekeo.eu/data).
The queries can be fund in the `data/jsons` folder
+ Global 10-daily Leaf Area Index 333m
```json
{
  "datasetId": "EO:CLMS:DAT:CGLS_GLOBAL_LAI300_V1_333M",
  "dateRangeSelectValues": [
    {
      "name": "dtrange",
      "start": "2022-06-01T00:00:00.000Z",
      "end": "2022-06-30T23:59:59.999Z"
    }
  ]
}
```
+ Level 2 Land - Sea and Land Surface
Temperature Radiometer (SLSTR) - Sentinel-3
```json
{
  "datasetId": "EO:ESA:DAT:SENTINEL-3:SL_2_LST___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        18.99804053609134,
        47.42120186691113,
        19.190237776905892,
        47.58048586099437
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2022-06-01T00:00:00.000Z",
      "end": "2022-06-30T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "productType",
      "value": "LST"
    },
    {
      "name": "timeliness",
      "value": "Near+Real+Time"
    },
    {
      "name": "orbitDirection",
      "value": "ascending"
    },
    {
      "name": "processingLevel",
      "value": "LEVEL2"
    }
  ]
}
```
+ Global 10-daily Fraction of Vegetation Cover 333m
```json
{
  "datasetId": "EO:CLMS:DAT:CGLS_GLOBAL_FCOVER300_V1_333M",
  "dateRangeSelectValues": [
    {
      "name": "dtrange",
      "start": "2022-06-01T00:00:00.000Z",
      "end": "2022-06-30T23:59:59.999Z"
    }
  ]
}
```
### External dataset
The square meter prices dataset was collected on 12th July 2022 using our [freely available scraper](https://github.com/Urbanum-Lab/budapest_property_price_scraper). The data is in the `data/aggragated` folder. **WARNING**: Check the README.md file of the scraper to get your own data.
## Outline
At the end of this notebook you will know:
+ How to aggregate data using the h3 library
+ How to visualize the data using the pydeck package
+ How to investigate
## Library imports
Here, we import packages for the project.

In [ ]:
import json
import os

import h3
import xarray as xr
from hda import Client


## Introduction
## Gathering the data
**WARNING**: Downloading the datasets takes time! The data will be downloaded into the current working directory. Use your favorite tools to move the downloaded files to the appropriate folders.
### SLSTR

In [6]:
c = Client(debug=True)
with open("../data/jsons/temperature.json") as infile:
    query = json.load(infile)
matches = c.search(query)
matches.download()

2022-07-18 10:27:37,075 INFO Download rate 2.2M/s   
2022-07-18 10:27:37,076 DEBUG {'downloadUri': None, 'filename': 'S3B_SL_2_LST____20220628T194550_20220628T194850_20220628T221819_0180_067_299_0720_PS2_O_NR_004.SEN3.zip', 'order': None, 'productInfo': {'datasetId': 'EO:ESA:DAT:SENTINEL-3:SL_2_LST___', 'product': 'S3B_SL_2_LST____20220628T194550_20220628T194850_20220628T221819_0180_067_299_0720_PS2_O_NR_004.SEN3', 'productEndDate': '2022-06-28T19:48:49.595000Z', 'productStartDate': '2022-06-28T19:45:49.595000Z'}, 'size': 72987415, 'url': '9cc5a076-8c00-5161-9c90-a0e6d384b2f5/S3B_SL_2_LST____20220628T194550_20220628T194850_20220628T221819_0180_067_299_0720_PS2_O_NR_004.SEN3.zip'}
2022-07-18 10:27:37,076 DEBUG ===> POST https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder
2022-07-18 10:27:37,077 DEBUG ===> POST {"jobId": "d2CLuZPhvqF1CTkRMfuwrbFaSZs", "uri": "9cc5a076-8c00-5161-9c90-a0e6...
2022-07-18 10:27:52,822 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443

HTTPError: 403 Client Error: FORBIDDEN for url: https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder/status/Xz0urIDhJf66dxlyiJLh6JkvIlg

### LAI

In [ ]:
c = Client(debug=True)
with open("../data/jsons/lai.json") as infile:
    query = json.load(infile)
matches = c.search(query)
matches.download()

### FCOVER

In [ ]:
c = Client(debug=True)
with open("../data/jsons/fcover.json") as infile:
    query = json.load(infile)
matches = c.search(query)
matches.download()

*Your operating system and tools might be different, below you can read tips which might be useful
on a Linux machine*
+ The notebook assumes that all data is in the `data` folder.
+ Move all zip and nc files
to the folder (e.g. you can use the `mv` command, like `mv ../data "*.zip"`)
+ Make folders for the data fiels (`cd data; mkdir leaf_data temp_data fcover`)
+ Move the *.nc files to the corresponding folders (e.g. move Leaf Area Index data into `leaf_data`)
+ Move the *.zip files into the `temp_data` folder `cd temp_data; unzip "*.zip"; rm "*.zip"`

## Cleaning and transforming data
